<a href="https://colab.research.google.com/github/seungmin-son/ML_Practice/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5%EB%A1%A0_7%EC%A3%BC%EC%B0%A8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요한 라이브러리, 데이터셋 통합

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mlxtend.plotting import scatterplotmatrix

In [51]:
red_wine = pd.read_csv('/content/winequality-red.csv', sep=';');
white_wine = pd.read_csv('/content/winequality-white.csv', sep=';'); #5주차에 사용하였던 wine dataset 사용

In [52]:
red_wine['color'] =1
white_wine['color'] = 0

In [53]:
red_wine.shape, white_wine.shape

((1599, 13), (4898, 13))

In [54]:
wine = pd.concat([red_wine,white_wine])
wine.shape

(6497, 13)

# 데이터 전처리

In [55]:
y = wine['color']
X = wine.drop(['color'], axis= 1)
X.shape

(6497, 12)

##Scaler 적용

In [56]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler()
scaler.fit(X)

X_scaled = scaler.transform(X)


## 데이터셋 분리

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size= 0.2,
                                                 random_state = 13)

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
log_reg  = LogisticRegression(random_state=13, solver='liblinear', C=10.)
log_reg.fit(X_train,y_train)

pred = log_reg.predict(X_test)
accuracy_score(y_test,pred)

0.9969230769230769

In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print('accuracy_score:  : ', accuracy_score(y_test,pred))
print('precision_score : ', precision_score(y_test, pred))
print('recall_score : ', recall_score(y_test, pred))

accuracy_score:  :  0.9969230769230769
precision_score :  0.9905362776025236
recall_score :  0.9968253968253968


#Binarizer 사용

In [60]:
from sklearn.preprocessing import Binarizer

thresholds = [0.05, 0.3, 0.5, 0.8, 0.95]

for thr in thresholds:
    pred_proba = log_reg.predict_proba(X_test)
    pred_proba_1 = pred_proba[:,1].reshape(-1,1)
    binarizer = Binarizer(threshold=thr).fit(pred_proba_1)
    custom_predict = binarizer.transform(pred_proba_1)

    accuracy = accuracy_score(y_test, custom_predict)
    precision = precision_score(y_test, custom_predict)
    recall = recall_score(y_test, custom_predict)

    print('Threshold: ', thr)
    print('accuracy_score: ', accuracy)
    print('precision_score: ', precision)
    print('recall_score: ', recall)
    print('\n')

Threshold:  0.05
accuracy_score:  0.9761538461538461
precision_score:  0.9127906976744186
recall_score:  0.9968253968253968


Threshold:  0.3
accuracy_score:  0.9946153846153846
precision_score:  0.98125
recall_score:  0.9968253968253968


Threshold:  0.5
accuracy_score:  0.9969230769230769
precision_score:  0.9905362776025236
recall_score:  0.9968253968253968


Threshold:  0.8
accuracy_score:  0.9946153846153846
precision_score:  0.9967741935483871
recall_score:  0.9809523809523809


Threshold:  0.95
accuracy_score:  0.9815384615384616
precision_score:  0.9965870307167235
recall_score:  0.926984126984127




전반적으로 로지스틱 회귀모델은 높은 accuracy, precision, recall값을 보인다.

그리고 Binarizer를 이용하여 비교해본결과
1. Threshold 0.05 : 임계값을 낮추면 모델이 Positive로 판단하는 기준이 낮아져 더 많은 샘플을 Positive로 예측하게 된다. 이 경우, 정밀도는 감소하지만 재현율은 증가하는 경향을 보인다.

2. Threshold 0.3, 0.5 : 임계값을 높여가며 정확도와 정밀도는 증가하고, 재현율은 거의 변화하지 않았다. 이는 모델이 Positive 클래스를 더욱 신중하게 예측하면서 성능이 개선되었음을 보여준다.

3. Threshold 0.8: 임계값을 더 높인 결과, 재현율이 약간 감소하였음, 이는 Positive로 예측하는 기준이 높아져서 FN(False Negative)이 증가했기 때문이다.

4. Threshold 0.95: 임계값을 더욱 높인 결과, 재현율이 크게 감소하였다. 이는 높은 임계값으로 인해 많은 Positive 케이스를 Negative로 판단하게 되어 FN이 크게 증가했기 때문이다.

결과적으로, 정밀도와 재현율은 반비례하는 경향을 보인다.